In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import seaborn as sns
import re
from os import path
from glob import glob
from tqdm import tqdm
import numpy as np
from matplotlib import cm
from pprint import pprint


In [3]:
StatisticsColumns = [
    'ReachedKnowledgeStep',
    #'AlmostReachedKnowledgeStep',
    'Matched_HorizontalPiece', 'Matched_VerticalPiece', 'Matched_CrossPiece', 'Matched_RocketPiece', 'Matched_BombPiece', 'Matched_RainbowPiece']

In [4]:
original_df = pd.read_csv('original_df.csv')
print(set(original_df['objective']))

{'kp', 'knowledge', '-', 'score'}


C:\Users\GIST\.conda\envs\mlagents\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (29,31) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
original_df.head()


,Unnamed: 0,EpisodeCount,StepCount,Time,InstanceUUID,SettleCount,ChangedCount,MeanComparisonCount,StdComparisonCount,ReachedKnowledgeStep,...,Target_BombPiece,Target_RainbowPiece,method,player,simpleEffect,objective,simulation,almostRatio,playerDepth,Sum_Matched
0,0,1,300,2023-05-21 04:18:54.310,7c6ae160,0,213,0.860000,0.669748,-1,...,2,3,mcts,0,1,knowledge,100,0.75,1,486
1,1,2,303,2023-05-21 04:21:26.273,7c6ae160,0,223,0.947195,0.712135,117,...,2,3,mcts,0,1,knowledge,100,0.75,1,414
2,2,3,300,2023-05-21 04:23:38.134,7c6ae160,0,234,0.913333,0.611338,94,...,2,3,mcts,0,1,knowledge,100,0.75,1,456
3,3,4,301,2023-05-21 04:25:35.225,7c6ae160,0,201,0.784053,0.640217,86,...,2,3,mcts,0,1,knowledge,100,0.75,1,354
4,4,5,301,2023-05-21 04:27:34.357,7c6ae160,0,203,0.790698,0.657809,121,...,2,3,mcts,0,1,knowledge,100,0.75,1,378


In [6]:
original_df.columns.values.tolist()

['Unnamed: 0',
 'EpisodeCount',
 'StepCount',
 'Time',
 'InstanceUUID',
 'SettleCount',
 'ChangedCount',
 'MeanComparisonCount',
 'StdComparisonCount',
 'ReachedKnowledgeStep',
 'Q1ReachedKnowledgeStep',
 'Q2ReachedKnowledgeStep',
 'Q3ReachedKnowledgeStep',
 'Matched_HorizontalPiece',
 'Matched_VerticalPiece',
 'Matched_CrossPiece',
 'Matched_RocketPiece',
 'Matched_BombPiece',
 'Matched_RainbowPiece',
 'Target_HorizontalPiece',
 'Target_VerticalPiece',
 'Target_CrossPiece',
 'Target_RocketPiece',
 'Target_BombPiece',
 'Target_RainbowPiece',
 'method',
 'player',
 'simpleEffect',
 'objective',
 'simulation',
 'almostRatio',
 'playerDepth',
 'Sum_Matched']

In [7]:
original_df = original_df[
    (original_df['ReachedKnowledgeStep'] != -1) &
    (original_df['Q1ReachedKnowledgeStep'] != -1) &
    (original_df['Q2ReachedKnowledgeStep'] != -1) &
    (original_df['Q3ReachedKnowledgeStep'] != -1)
]
original_df.rename(columns={
                            'ReachedKnowledgeStep': '100%',
                            'Q1ReachedKnowledgeStep': '25%',
                            'Q2ReachedKnowledgeStep': '50%',
                            'Q3ReachedKnowledgeStep': '75%'
                            }, inplace=True)


for i in range(5):
    original_df['playerDepth'] = original_df['playerDepth'].replace(str(i), i)
for i in range(100, 1000, 100):
    original_df['simulation'] = original_df['simulation'].replace(str(i), i)

In [8]:
pd.set_option('display.max_rows', None)
match_result = original_df.groupby(['player', 'method', 'objective', 'playerDepth', 'almostRatio', 'simulation'])[
                                      'method',
                                      'Matched_HorizontalPiece',
                                      'Matched_VerticalPiece',
                                      'Matched_CrossPiece',
                                      'Matched_RocketPiece',
                                      'Matched_BombPiece',
                                      'Matched_RainbowPiece',
                                      #'AlmostReachedKnowledgeStep',
                                      '25%',
                                      '50%',
                                      '75%',
                                      '100%',
                                      'Sum_Matched'].agg(
    {
        'method': ['count'],
        #'AlmostReachedKnowledgeStep': ['mean'],
        '25%': ['mean'],
        '50%': ['mean'],
        '75%': ['mean'],
        '100%': ['mean'],
        'Matched_HorizontalPiece':['mean'],
        'Matched_VerticalPiece':['mean'],
        'Matched_CrossPiece':['mean'],
        'Matched_RocketPiece':['mean'],
        'Matched_BombPiece':['mean'],
        'Matched_RainbowPiece':['mean'],
        'Sum_Matched':['mean'],
     }
)
match_result


C:\Users\GIST\.conda\envs\mlagents\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


method         25%  \
                                                            count        mean   
player method objective playerDepth almostRatio simulation                      
0      mcts   knowledge 1           0.75        100           485   70.004124   
                                                200           484   68.522727   
                                                400           491   69.564155   
                                    1.0         100           477   71.503145   
                                                200           482   69.875519   
                                                400           492   68.898374   
                        2           0.75        100           485   74.061856   
                                                200           485   71.546392   
                                                400           487   69.388090   
                                    1.0         100           483   69.991718   
                                                200           486   72.026749   
                                                400           482   66.858921   
              kp        1           -           100           491   72.431772   
                                                200           485   65.195876   
                                                400           487   69.517454   
                        2           -           100           346   73.393064   
                                                200           302   70.331126   
                                                400           290   68.758621   
              score     1           -           100           483   69.291925   
                                                200           484   67.729339   
                                                400           490   64.767347   
                        2           -           100           484   72.264463   
                                                200           486   69.327160   
                                                400           482   71.394191   
       random -         -           -           -             414   93.514493   
1      mcts   knowledge 1           0.75        100           449   78.966592   
                                                200           444   81.984234   
                                                400           444   77.358108   
                                    1.0         100           440   82.002273   
                                                200           453   79.560706   
                                                400           453   79.986755   
                        2           0.75        100           462   83.714286   
                                                200           440   84.590909   
                                                400           449   84.013363   
                                    1.0         100           436   85.183486   
                                                200           461   79.251627   
                                                400           446   84.609865   
              kp        1           -           100           442   81.457014   
                                                200           445   80.734831   
                                                400           463   76.665227   
                        2           -           100           330   82.900000   
                                                200           283   80.328622   
                                                400           263   83.250951   
              score     1           -           100           462   84.878788   
                                                200           469   79.857143   
                                                400           448   84.299107   
                        2           -           100           432   86.724537   
         

In [19]:
# pd.set_option('display.max_rows', None)


match_result = original_df.copy()
match_result = match_result[~match_result['almostRatio'].isin(['0.75'])]
match_result = match_result[~match_result['playerDepth'].isin([2])]
match_result = match_result[~match_result['simulation'].isin([100, 400])]



match_result = match_result.groupby(['method', 'objective'])[
                                      '25%',
                                      '50%',
                                      '75%',
                                      '100%',
                                      'Sum_Matched'].agg(
    {
        '25%': ['mean'],
        '50%': ['mean'],
        '75%': ['mean'],
        '100%': ['mean'],
     }
)
match_result

C:\Users\GIST\.conda\envs\mlagents\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  if sys.path[0] == "":


25%         50%         75%        100%
                        mean        mean        mean        mean
method objective                                                
mcts   knowledge   78.380312  102.269500  135.472835  145.286713
       kp          76.814775   98.539924  132.038566  141.712656
       score       77.614156  102.043108  132.852581  142.086216
random -          104.224753  131.037206  171.634776  181.931663

In [21]:
for i in range(0, 4):
    match_result = original_df.copy()
    match_result = match_result[match_result['player'] == i]
    match_result = match_result[~match_result['almostRatio'].isin(['0.75'])]
    match_result = match_result[~match_result['playerDepth'].isin([2])]
    match_result = match_result[~match_result['simulation'].isin([100, 400])]



    match_result = match_result.groupby(['method', 'objective'])[
                                          '25%',
                                          '50%',
                                          '75%',
                                          '100%',
                                          'Sum_Matched'].agg(
        {
            '25%': ['mean'],
            '50%': ['mean'],
            '75%': ['mean'],
            '100%': ['mean'],
         }
    )
    print(match_result)


                        25%         50%         75%        100%
                       mean        mean        mean        mean
method objective                                               
mcts   knowledge  69.875519   85.657676  117.661826  117.883817
       kp         65.195876   80.037113  111.649485  111.830928
       score      67.729339   84.834711  114.152893  114.518595
random -          93.514493  112.314010  153.678744  154.217391
                         25%         50%         75%        100%
                        mean        mean        mean        mean
method objective                                                
mcts   knowledge   79.560706  115.185430  153.916115  165.015453
       kp          80.734831  114.737079  149.035955  159.233708
       score       79.857143  119.277186  152.658849  160.277186
random -          100.042146  146.360153  190.478927  202.463602
                         25%         50%         75%        100%
                        mean    

C:\Users\GIST\.conda\envs\mlagents\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  # This is added back by InteractiveShellApp.init_path()
C:\Users\GIST\.conda\envs\mlagents\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  # This is added back by InteractiveShellApp.init_path()
C:\Users\GIST\.conda\envs\mlagents\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  # This is added back by InteractiveShellApp.init_path()
C:\Users\GIST\.conda\envs\mlagents\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  # This i

In [17]:
top = cm.get_cmap('Oranges_r', 512)
bottom = cm.get_cmap('Blues', 512)
newcolors = np.vstack((top(np.linspace(0, 1, 512)),
                       bottom(np.linspace(0, 1, 512))))
newcolors = newcolors[:, :3]

MIN = 0
MAX = 1

cmap = plt.cm.get_cmap('RdBu_r')

def rgb_to_hex(r, g, b):
    r, g, b = int(r * 255), int(g * 255), int(b * 255)
    return hex(r)[2:].zfill(2) + hex(g)[2:].zfill(2) + hex(b)[2:].zfill(2)

def styler_2(col):

    cmap = plt.cm.get_cmap('RdBu_r')
    means = col.groupby(level=0).transform('min')

    if col.name[0] != 'Result[0]':

        text_bold = ((col <= means).map({
            True: 'textbf:--rwrap;',
            False: ''
        })).to_list()
    else:
        text_bold = ['' for _ in col]

    colors = list()
    for row, bf_str in zip(col, text_bold):

        font_color = 'white' if abs(row) > 0.4 else 'black'

        _val = row - 0.15
        _val = _val + 0.5
        hex_code = rgb_to_hex(*list(cmap(_val))[:3])

        tag = bf_str
        colors.append(tag)
    return colors


styled_table = match_result.style.apply(styler_2).format(precision=3)

with open('required_steps.tex', 'w') as f:
    latex_str = styled_table.to_latex(position_float='centering',
                                      multicol_align="c",
                                      label='tab:pcg_controllbility_small',
                                      caption='Descriptive statistics on the controllability of the generative models',
                                      hrules=True)
    latex_str = latex_str.replace('\\begin{table}', '\\begin{table}[!h]')
    latex_str = latex_str.replace('%', '\\%')


    latex_list = latex_str.splitlines()

    # latex_list = concat_mean_std(latex_list, start_row_num=10, end_row_num=31, column_num=2)
    # latex_list = concat_mean_std(latex_list, start_row_num=10, end_row_num=31, column_num=3)
    #
    # i = 13
    # while i < len(latex_list) - 4:
    #     latex_list.insert(i, '\midrule')
    #     i += 4

    latex_str = '\n'.join(latex_list)
    f.write(latex_str)



In [ ]:
def get_legend_name(x):
    if x['method'] == 'mcts':
        if x['objective'] == 'score':
            return f"{x['method']}_{x['objective']}_{x['simulation']}_{x['playerDepth']}"
        elif x['objective'] == 'knowledge':
            return f"{x['method']}_{x['objective']}_{x['almostRatio']}_{x['simulation']}_{x['playerDepth']}"
        elif x['objective'] == 'kp':
            return f"{x['method']}_{x['objective']}_{x['almostRatio']}_{x['simulation']}_{x['playerDepth']}"
    else:
        return f"{x['method']}"

In [ ]:

melted_df = original_df.copy()
#
# melted_df.rename(columns={
#     'ReachedKnowledgeStep':'100',
#     'Q1ReachedKnowledgeStep':'25',
#     'Q2ReachedKnowledgeStep':'50',
#     'Q3ReachedKnowledgeStep':'75'
# }, inplace=True)

_columns = [
'25%', '50%', '75%', '100%',
# Matched_HorizontalPiece', 'Matched_VerticalPiece', 'Matched_CrossPiece', 'Matched_RocketPiece', 'Matched_BombPiece', 'Matched_RainbowPiece'
]

# print(player_df.keys())
sns.set_palette('Set2')
# melted_df = melted_df.groupby(['method', 'objective', 'playerDepth', 'simulation']).sample(n=400, random_state=1, replace=True)
melted_df['concat_method'] = melted_df.agg(lambda x: get_legend_name(x), axis=1)
melted_df = pd.melt(melted_df, id_vars=['EpisodeCount', 'objective', 'concat_method', 'simulation', 'player'],
                    value_vars=[*_columns], ignore_index=False)
# pprint(set(melted_df['concat_method']))
_methods = [
    'mcts_kp_-_200_1',
    'mcts_knowledge_1.0_200_1',
    'mcts_score_200_1',
    'random'
]
melted_df = melted_df[melted_df['concat_method'].isin(_methods)]

plt.figure(figsize=(5, 4))
plt.clf()
ax = sns.boxplot(x="variable", y="value", hue="concat_method", data=melted_df, dodge=True, linewidth=0.8)

plt.legend(loc='upper left')
ax.set_title(f'Required Steps for Learning Skill')
ax.set_xlabel('Progress (%)')
ax.set_ylabel('Step')
# plt.savefig(f'./figures/learning_steps_p{player_no}', dpi=300)
plt.show()